In [16]:
import pandas as pd
import os
import requests

file_name = 'ipf_lifts.csv'
document_url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-10-08/ipf_lifts.csv'  

if os.path.exists(file_name):
    pass
else:
    response = requests.get(document_url)
    if response.status_code == 200:
        with open(file_name, 'wb') as f:
            f.write(response.content)
    else:
        print(f'Failed to download the document. Status code: {response.status_code}')

lifts = pd.read_table('ipf_lifts.csv', sep=",", encoding="utf8", engine='python')

df_lifts = pd.read_csv('ipf_lifts.csv')
print(df_lifts.sample(n = 5)) 

                         name sex event   equipment   age age_class  \
36622             E. Richards   F   SBD  Single-ply   NaN       NaN   
6357   Anastasiya Potrashkova   F   SBD         Raw  18.5     18-19   
9314            Kendra Miller   F   SBD  Single-ply  15.0     13-15   
11883          Bernhard Boyer   M   SBD  Single-ply  41.5     40-44   
20993             Rex Parsons   M   SBD         Raw  21.5     20-23   

          division  bodyweight_kg weight_class_kg  best3squat_kg  \
36622      Juniors          51.55              52           95.0   
6357       Juniors          62.01              63          127.5   
9314   Sub-Juniors          51.50              52          145.0   
11883    Masters 1         100.00             100          255.0   
20993         Open         104.76             105          232.5   

       best3bench_kg  best3deadlift_kg place        date federation  \
36622           47.5             105.0    11  1992-09-08        IPF   
6357            82.5  

In [21]:
# 2. Знайдіть рекорди кожної статі та кожного підрозділу в кожній із 3 вправ (жим лежачи, присідання та станова тяга).
# жим лежачи
bench_press_records = df_lifts.groupby(['sex' , 'division']).agg({'best3bench_kg': 'max'})
print(bench_press_records)

# присідання
squats = df_lifts.groupby(['sex' , 'division']).agg({'best3squat_kg': 'max'})
print(squats)

# станова тяга
dedlift = df_lifts.groupby(['sex' , 'division']).agg({'best3deadlift_kg': 'max'})
print(dedlift)


In [32]:
# 3. Порахуйте кількість перемог кожного учасника, беручи за перемогу 1-е місце. Збережіть результати у вторинному DataFrame
df_lifts['place'] = pd.to_numeric(df_lifts['place'], errors='coerce')

winners = df_lifts[df_lifts['place'] == 1].groupby('name').agg(winners_number=('place', 'count'))
print (winners)

df = df_lifts.merge(winners, how='left', on='name')
print(df)


                           winners_number
name                                     
A Ernandos-Ortega                       1
A. Raface                               1
Aarre Käpylä                            2
Ab Wolders                              3
Abdulkarim Karaman                      4
...                                   ...
Éva Makrai                              2
Øyvind Bjørnsen                         1
Þórunn Brynja Jónasdóttir               1
Štefan Koľšovský                        7
Štěpánka Židová                         1

[3328 rows x 1 columns]
                    name sex event   equipment   age age_class division  \
0       Hiroyuki Isagawa   M   SBD  Single-ply   NaN       NaN      NaN   
1        David Mannering   M   SBD  Single-ply  24.0     24-34      NaN   
2          Eddy Pengelly   M   SBD  Single-ply  35.5     35-39      NaN   
3       Nanda Talambanua   M   SBD  Single-ply  19.5     20-23      NaN   
4        Göran Henrysson   M   SBD  Single-ply   NaN  

In [33]:
'''4. Використайте DataFrame з вправи 3, щоб показати для кожної комбінації 
підрозділу та статі, учасника з найбільшою кількістю перемог серед учасників, які 
будь-коли брали участь у цьому дивізіоні'''
division_best = df.groupby(['division', 'weight_class_kg', 'sex']).winners_number.max()
print(division_best)

division    weight_class_kg  sex
Heavy       100              M       9.0
            105              M      11.0
            67.5+            F      22.0
            72               F      17.0
            75               F      17.0
                                    ... 
Super       75+              F      22.0
SuperHeavy  120              M       6.0
            120+             M       8.0
            84               F       4.0
            84+              F      22.0
Name: winners_number, Length: 268, dtype: float64
